# Data Cleanup and Transformation

## Open CSV data for cleanup

In [1]:
import pandas as pd
df = pd.read_csv("../data/doi_companies_pdf.csv")

In [2]:
df.head()

,sn,register_date,investor,name,address,district,total_capital,fixed_capital,working_capital,annual_capacity,employment,category,scale,power,investment
0,1,43327,NaN,HIMAL IRON & STEEL PVT.LTD,पसा िज ला,PARSA,364369000,345360000,19009000,IRON ROD 75000 MT,0,MANUFACTURING,LARGE,3975 KW,Local - 100%
1,2,44620,CHANDRA MAN GURUNG,KALPANA CINEMA HALL,का क िज ला,KASKI,40000000,34600000,5400000,"ENTTERTAINMENT, FILM SHOWS 805 SEATS",21,SERVICE,MEDIUM,0,Local - 100%
2,3,44640,RAM SHANKER SHRESHTA,SHANKATA HOTEL,काठमाड िज ला,KATHMANDU,0,0,0,"HOTEL, 135 ROOMS",40,TOURISM,SMALL,10,Local - 100%
3,4,47553,ी वस त कुमार चौधर समेत,PASHUPATI BISCUITS INDUSTRIES PVT. LTD.,स नु स र िज ला दहु वी गा. व.स. वडा न.ं ७,SUNSARI,200000000,148000000,52000000,"BISCUIT 2700 M.T., CHOCOLATE CANDY 600 M.T., B...",165,MANUFACTURING,SMALL,1000 KVA,Local - 100%
4,5,47671,NaN,NARAYANI HOTEL PVT. LTD.,ल लतपरु िज ला,LALITPUR,74749000,70000000,4749000,HOTEL SERVICE 113 ROOMS,189,TOURISM,MEDIUM,40 KW,Local - 100%


In [ ]:
df.drop()

## Extract year from date

Lets try with date of first row. Lets see all the fields of the first row using iloc

In [4]:
df.iloc[0]

sn                                          1
register_date                           43327
investor                                  NaN
name               HIMAL IRON & STEEL PVT.LTD
address                             पसा िज ला
district                                PARSA
total_capital                       364369000
fixed_capital                       345360000
working_capital                      19009000
annual_capacity             IRON ROD 75000 MT
employment                                  0
category                        MANUFACTURING
scale                                   LARGE
power                                 3975 KW
investment                       Local - 100%
Name: 0, dtype: object

In [5]:
df.iloc[0]['register_date']

'43327'

We can just extract the date field from the first row.

In [ ]:
??

We follow the instruction given in stackoverflow to convert excel date to python datetime

https://stackoverflow.com/questions/38454403/convert-excel-style-date-with-pandas

In [6]:
import datetime as dt
dateval = pd.TimedeltaIndex([int(43327)],unit='d') + dt.datetime(1899, 12, 30)
dateval

DatetimeIndex(['2018-08-15'], dtype='datetime64[ns]', freq=None)

In [10]:
df.dtypes

sn                  int64
register_date      object
investor           object
name               object
address            object
district           object
total_capital      object
fixed_capital      object
working_capital     int64
annual_capacity    object
employment          int64
category           object
scale              object
power              object
investment         object
dtype: object

Lets apply this to all the rows.

In [11]:
pd.TimedeltaIndex([pd.to_numeric(df["register_date"])],unit='d') + dt.datetime(1899, 12, 30)

ValueError: Unable to parse string "2069-04-31" at position 4872

There are "2069-04-31" dates in the rows. Excel didn't convert these because this is invalid english date.

Lets identity the rows with such date "2069-04-31".

In [12]:
df[df["register_date"].str.contains("-")==True]["register_date"].head()

4872    2069-04-31
4873    2069-04-32
4874    2069-04-32
4875    2069-04-32
5231    2070-02-29
Name: register_date, dtype: object

Since this is string (object), we can now extract the first 4 char (year) from the above values.

In [13]:
df[df["register_date"].str.contains("-")==True]["register_date"].str[0:4]

4872    2069
4873    2069
4874    2069
4875    2069
5231    2070
5232    2070
5233    2070
5234    2070
5313    2070
5602    2071
5603    2071
5643    2071
5644    2071
5645    2071
5646    2071
5768    2071
6041    2072
6042    2072
6043    2072
6044    2072
6045    2072
6046    2072
6047    2072
6048    2072
6049    2072
6050    2072
6051    2072
6141    2072
6454    2073
6455    2073
6456    2073
6457    2073
6458    2073
6459    2073
Name: register_date, dtype: object

In [16]:
df[df["register_date"].str.contains("-")==True]["register_date"].head().str[:4]

4872    2069
4873    2069
4874    2069
4875    2069
5231    2070
Name: register_date, dtype: object

Now we can find the dates with numeric values.

In [17]:
print(df[df["register_date"].str.contains("-")==False]["register_date"].head(3))

0    43327
1    44620
2    44640
Name: register_date, dtype: object


Now lets create a new column "register_year" following the above code.

In [27]:
import datetime as dt
dateval = pd.TimedeltaIndex(pd.to_numeric(df[df["register_date"].str.contains("-")==False]["register_date"]), unit = "d")  + dt.datetime(1899, 12, 30)

#print(dateval)
df.loc[df["register_date"].str.contains("-")==False,"register_year"] = dateval.year
df.loc[df["register_date"].str.contains("-")==True,"register_year"] = pd.to_numeric(df[df["register_date"].str.contains("-")==True]["register_date"].str[:4])

#df.dtypes
#df['register_year'].value_counts()
print(df.groupby("register_year").size())
#print(type(dateval))

register_year
2018.0      1
2022.0      2
2030.0      4
2032.0      1
2033.0      2
2034.0      2
2035.0      2
2037.0      2
2038.0      1
2039.0      5
2040.0      1
2041.0      2
2042.0      3
2043.0      5
2044.0      4
2045.0      6
2046.0      4
2047.0     96
2048.0    177
2049.0    878
2050.0    127
2051.0    193
2052.0    335
2053.0    290
2054.0    147
2055.0     95
2056.0    146
2057.0    162
2058.0    144
2059.0    104
2060.0    149
2061.0    108
2062.0    121
2063.0    140
2064.0    224
2065.0    276
2066.0    275
2067.0    241
2068.0    260
2069.0    393
2070.0    374
2071.0    471
2072.0    390
2073.0    156
dtype: int64


In [28]:
import datetime as dt
dateval = pd.TimedeltaIndex(pd.to_numeric(df[df["register_date"].str.contains("-")==False]["register_date"]), unit = "d")  + dt.datetime(1899, 12, 30)

#print(dateval)
df.loc[df["register_date"].str.contains("-")==False,"register_month"] = dateval.month
df.loc[df["register_date"].str.contains("-")==True,"register_month"] = pd.to_numeric(df[df["register_date"].str.contains("-")==True]["register_date"].str[5:7])

#df.dtypes
#df['register_year'].value_counts()
print(df.groupby("register_month").size())
#print(type(dateval))

register_month
1.0     544
2.0     719
3.0     739
4.0     613
5.0     611
6.0     553
7.0     556
8.0     472
9.0     411
10.0    416
11.0    396
12.0    489
dtype: int64


In [33]:
print(df.groupby(["register_year","register_month"]).size())

register_year  register_month
2018.0         8.0                1
2022.0         2.0                1
               3.0                1
2030.0         3.0                1
               7.0                2
                                 ..
2072.0         11.0              23
               12.0              33
2073.0         1.0               36
               2.0               61
               3.0               59
Length: 356, dtype: int64


## Check District names

Lets do quick summary of district names.

In [34]:
df.groupby("district").size()

district
ACHHAM          4
ARGHAKHACHI     3
BAGLUNG         8
BAITADI         6
BAJHANG         4
               ..
SYANGJA         4
TANAHU         18
TAPLEJUNG      11
TEHRATHUM       5
UDAYPUR         5
Length: 74, dtype: int64

We don't see all the names. Lets display full list. 

In [35]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')
print_full(df.groupby("district").size())

district
ACHHAM                4
ARGHAKHACHI           3
BAGLUNG               8
BAITADI               6
BAJHANG               4
BAJURA                1
BANKE                72
BARA                214
BARDIYA              16
BHAKTAPUR           151
BHOJPUR               7
CHITWAN             176
DADELDHURA            5
DAILEKH               4
DANG                 35
DARCHULA              6
DHADING              51
DHANKUTA             17
DHANUSHA             33
DOLKHA               33
DOLPA                 1
DOTI                  6
GORKHA               24
GULMI                 5
HUMLA                 5
ILAM                 38
JAJARKOT              3
JHAPA               104
KAILALI              36
KALIKOT               4
KANCHANPUR           28
KAPILBASTU           31
KASKI               333
KATHMANDU          2931
KAVRE               139
KHOTANG               4
LALITPUR            728
LAMJUNG              26
MAHOTTARI             5
MAKWANPUR           103
MANANG                1
MORANG 

In [36]:
df["district"].nunique()

74

All districts' names look ok here. There are few spelling mistakes but we can live with that for now.

## Check "Total Capital"

"Total Capital" should be of numeric type. Lets see the types of all the fields.

### Show field types

In [37]:
df.dtypes

sn                   int64
register_date       object
investor            object
name                object
address             object
district            object
total_capital       object
fixed_capital       object
working_capital      int64
annual_capacity     object
employment           int64
category            object
scale               object
power               object
investment          object
register_year      float64
register_month     float64
dtype: object

We see that total_capital is "object" type, which is not numeric. See https://pbpython.com/pandas_dtypes.html for different panda datatypes used.

### Converting "total capital" to numeric

In [38]:
df["total_capital"] = pd.to_numeric(df["total_capital"])

ValueError: Unable to parse string "############" at position 4247

There are ## characters in the "Total Capital" column. Lets see what are those. Lets see the rows containing ## characters in the "Total Capital"

In [39]:
df[df["total_capital"].str.contains("##")].head(3)

,sn,register_date,investor,name,address,district,total_capital,fixed_capital,working_capital,annual_capacity,employment,category,scale,power,investment,register_year,register_month
4247,4248,61024,SEMC WEST SETI HOLDINGS,WEST SETI HYDRO PVT.LTD,डोट िज ला,DOTI,############,############,100000000,HYDROPOWER 750 MW,44,ENERGY BASED,LARGE,1500 KVA,"Local - 100%, Foreign - 0%",2067.0,1.0
4457,4458,61333,MS SAGARMATHA HYDRO POWER,SINO HYDRO SAGARMATHA POWER CO.PVT.LTD,लमजङु िज ला,LAMJUNG,############,############,22800000,HYDROPOWER 50 MWH,70,ENERGY BASED,LARGE,300 KVA,"Local - 100%, Foreign - 0%",2067.0,12.0
4546,4547,61444,M/S NEPAL BIDHYUT PRADHIKARAN,MATHILLO TAMAKOSHI HYDROPOWER LTD,दोलखा िज ला,DOLKHA,############,############,119000000,HYDROPOWER 456 MW,2335,ENERGY BASED,LARGE,0,Local - 100%,2068.0,3.0


These must have been issues when creating pdf from excel. For us, they are missing values. Either we could use 0 or employ other techniques to fill those missing values. For simplicity, lets fill those with 0.

In [40]:
df.loc[df["total_capital"].str.contains("#"), "total_capital"] = 0

Now lets try converting to numeric type.

In [41]:
df["total_capital"] = pd.to_numeric(df["total_capital"])

Lets calculate the sum of "total capital". We should get proper numeric value.

In [42]:
df["total_capital"].sum()

812573988508

In [48]:
df["employment"]

0         0
1        21
2        40
3       165
4       189
       ... 
6514     24
6515     41
6516     34
6517     53
6518     89
Name: employment, Length: 6519, dtype: int64

## Check employment values

Lets see the type of "employment" column

In [49]:
df["employment"].dtypes

dtype('int64')

It's int64, which means it's numeric column.

In [50]:
df["employment"]

0         0
1        21
2        40
3       165
4       189
       ... 
6514     24
6515     41
6516     34
6517     53
6518     89
Name: employment, Length: 6519, dtype: int64

In [51]:
df["employment"].sum()

520972

## Check category

Category should be strings or object type. But we just need to make sure that there are no spelling mistakes.

In [62]:
df["category"].value_counts()

MANUFACTURING              2640
SERVICE                    1804
TOURISM                    1300
AGRO AND FORESTRY BASED     395
ENERGY BASED                271
MINERAL                      61
CONSTRUCTION                 48
Name: category, dtype: int64

In [63]:
grouped = df.groupby("category")
grouped["category"].count()

category
AGRO AND FORESTRY BASED     395
CONSTRUCTION                 48
ENERGY BASED                271
MANUFACTURING              2640
MINERAL                      61
SERVICE                    1804
TOURISM                    1300
Name: category, dtype: int64

## Check scale

"Scale" is also of object type. Lets quickly check the summary of "Scale" values.

In [64]:
df.groupby("scale").size()

scale
LARGE      854
MEDIUM    1481
SMALL     4184
dtype: int64

## Split investment into local and foreign

"Investment" column contains the data in the following format
* Local - 100%
* Local - 64.11%, Foreign - 35.89%
* Foreign - 100%

We can't do anything with this column in its current form. But if we could separate local and foreign investment into 2 different columns, that could help us in various analysis. 

**Question for discussion**

* How would you approach this problem of splitting into 2 columns as shown above?

In [65]:
df.investment

0                           Local - 100%
1                           Local - 100%
2                           Local - 100%
3                           Local - 100%
4                           Local - 100%
                      ...               
6514    Local - 64.11%, Foreign - 35.89%
6515                      Foreign - 100%
6516                        Local - 100%
6517                        Local - 100%
6518                        Local - 100%
Name: investment, Length: 6519, dtype: object

### New column for local and foreign percentage

Lets use regex to extract Local percentage value and create new column.

In [71]:
df["local_percent"] = df.investment.str.extract("Local - ([0-9.]*).*")
df["local_percent"].fillna(0,inplace=True)
df.local_percent = pd.to_numeric(df.local_percent)
df.local_percent

0       100.00
1       100.00
2       100.00
3       100.00
4       100.00
         ...  
6514     64.11
6515      0.00
6516    100.00
6517    100.00
6518    100.00
Name: local_percent, Length: 6519, dtype: float64

In [72]:
df["foreign_percent"] = df.investment.str.extract(".*Foreign - ([0-9.]*).*")
df["foreign_percent"].fillna(0, inplace=True)
df.foreign_percent = pd.to_numeric(df.foreign_percent)
df.foreign_percent

0         0.00
1         0.00
2         0.00
3         0.00
4         0.00
         ...  
6514     35.89
6515    100.00
6516      0.00
6517      0.00
6518      0.00
Name: foreign_percent, Length: 6519, dtype: float64

## Final data types of cleaned data

In [73]:
df.dtypes

sn                   int64
register_date       object
investor            object
name                object
address             object
district            object
total_capital        int64
fixed_capital       object
working_capital      int64
annual_capacity     object
employment           int64
category            object
scale               object
power               object
investment          object
register_year      float64
register_month     float64
local_percent      float64
foreign_percent    float64
dtype: object

In [75]:
df[["investment","local_percent","foreign_percent"]]

,investment,local_percent,foreign_percent
0,Local - 100%,100.00,0.00
1,Local - 100%,100.00,0.00
2,Local - 100%,100.00,0.00
3,Local - 100%,100.00,0.00
4,Local - 100%,100.00,0.00
...,...,...,...
6514,"Local - 64.11%, Foreign - 35.89%",64.11,35.89
6515,Foreign - 100%,0.00,100.00
6516,Local - 100%,100.00,0.00
6517,Local - 100%,100.00,0.00


## Write to CSV

Lets write clean dataframe to file for exploration later.

In [76]:
df.to_csv("clean.csv", index = False)

Now we will explore the data for sights.